In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Final STAG AML data after all cleaning

In [ ]:
adata = sc.read("STAG_AML_final.h5ad")

In [ ]:
# sc.tl.leiden(adata, resolution=1.2) more granularity to tease out LMPP probably

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)
sc.pl.umap(
    adata,
    color=["leiden"],
    s=50,
    frameon=False,
    show=False,
    legend_loc="on data",
    legend_fontsize=20,
    legend_fontoutline=4,
)
plt.savefig(f"STAG_AML_final_leiden.jpg", bbox_inches="tight")

# do DE on leiden clusters

In [ ]:
adata.uns["log1p"]["base"] = None
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")

In [ ]:
result = adata.uns["rank_genes_groups"]
groups = result["names"].dtype.names
deg = pd.DataFrame(
    {
        group + "_" + key[:4]: result[key][group]
        for group in groups
        for key in ["names", "pvals_adj", "logfoldchanges"]
    }
)

In [ ]:
i = 6
deg_sub = deg[[f"{i}_name", f"{i}_pval", f"{i}_logf"]].copy()
deg_sub["logpval_mult_logf"] = -np.log10(deg_sub[f"{i}_pval"] + 10 ** (-300)) * np.abs(
    deg_sub[f"{i}_logf"]
)

deg_sub = (
    deg_sub[(deg_sub[f"{i}_pval"] < 0.001) & (deg_sub[f"{i}_logf"] > 1.5)]
    .sort_values(by=f"{i}_logf", ascending=False)
    .head(20)
)
deg_sub.head(20)

In [ ]:
genes = deg_sub[deg_sub.columns[0]].tolist()[:20]

In [ ]:
plt.rcParams["figure.figsize"] = (5, 5)
sc.pl.umap(
    adata,
    color=genes,
    s=20,
    frameon=False,
    show=False,
    legend_loc="on data",
    legend_fontsize=10,
    ncols=5,
)
plt.savefig(f"STAG_AML_final_DEGs_leiden_c{i}.jpg", bbox_inches="tight")

# name leiden clusters based on DEGs

In [ ]:
leiden_cell = {
    "0": "HSC",
    "1": "Cycling",
    "2": "GMP",
    "3": "GMP",
    "4": "HSC",
    "5": "CD14+CD16+ Mono.",
    "6": "Early Mono.",
    "7": "Naive CD4_CD8 T",
    "8": "Neutrophil",
    "9": "HSC",
    "10": "Neutrophil",
    "11": "NK_Eff_Mem T",
    "12": "CD16 Mono.",
    "13": "MEP",
    "14": "Erythrocytes",
    "15": "Cycling",
}

In [ ]:
adata.obs["cell_type"] = adata.obs.leiden.apply(lambda x: leiden_cell[x])

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)
sc.pl.umap(
    adata,
    color=["cell_type"],
    s=50,
    frameon=False,
    show=False,
    legend_loc="on data",
    legend_fontsize=20,
    legend_fontoutline=4,
)
plt.savefig(f"STAG_AML_final_cell_types.jpg", bbox_inches="tight")

# do DE on cell_type clusters

In [ ]:
adata.uns["log1p"]["base"] = None
sc.tl.rank_genes_groups(
    adata, "cell_type", method="wilcoxon", key_added="cell_type_DEGs"
)

In [ ]:
result = adata.uns["cell_type_DEGs"]
groups = result["names"].dtype.names
deg = pd.DataFrame(
    {
        group + "_" + key[:4]: result[key][group]
        for group in groups
        for key in ["names", "pvals_adj", "logfoldchanges"]
    }
)

In [ ]:
cell_types = adata.obs.cell_type.cat.categories

In [ ]:
len(cell_types), cell_types

In [ ]:
adata.write_h5ad("STAG_AML_annotated_DEGs.h5ad", compression="gzip")

# aggregate all cell_type DEG in one dictionary

In [ ]:
ALL_DEGs = {}
for i in range(len(cell_types)):
    i = cell_types[i]
    deg_sub = deg[[f"{i}_name", f"{i}_pval", f"{i}_logf"]].copy()
    deg_sub["logpval_mult_logf"] = -np.log10(
        deg_sub[f"{i}_pval"] + 10 ** (-300)
    ) * np.abs(deg_sub[f"{i}_logf"])

    deg_sub = deg_sub[
        (deg_sub[f"{i}_pval"] < 0.000001) & (deg_sub[f"{i}_logf"] > 1.5)
    ].sort_values(by=f"{i}_logf", ascending=False)
    ALL_DEGs[i] = deg_sub.head(20)

# plot tops DEGs of each cell_type cluster

In [ ]:
for c in range(len(cell_types)):
    col = 5
    row = 4
    wid = 5
    deg_sub = ALL_DEGs[cell_types[c]]
    genes = deg_sub[deg_sub.columns[0]].tolist()
    fig, ax = plt.subplots(
        row,
        col,
        figsize=(col * wid, row * wid),
        gridspec_kw={"wspace": 0.01, "hspace": 0.1},
    )
    axr = ax.ravel()
    for i, g in enumerate(genes):
        sc.pl.umap(adata, color=g, frameon=False, s=10, ax=axr[i], show=False)
        axr[i].set_title(f"{g}", fontsize=15)
        cbar = axr[i].collections[0].colorbar
        if cbar != None:
            cbar.remove()
    for j in np.arange(i + 1, len(axr)):
        axr[j].remove()
    plt.savefig(f"STAG_AML_final_DEGs_{cell_types[c]}.jpg", bbox_inches="tight")

# plot tops DEGs from the entire AML dataset on top of our dataset

In [ ]:
deg_full_data = pd.read_csv("../all_csvs/LMPP_DEG.csv", index_col=0)

In [ ]:
deg_full_data = pd.read_csv("../all_csvs/GMP_DEG.csv" ,index_col=0)

In [ ]:
deg_full_data = pd.read_csv("../all_csvs/Prog_Mk_DEG.csv", index_col=0)

In [ ]:
genes = adata.var[adata.var.index.isin(deg_full_data.index[:5])].index

In [ ]:
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(adata, color=genes, s=20, frameon=False)

# find which clusters have genes of interest as DEG in the entire AML dataset

In [ ]:
deg_full_data = pd.read_csv("../all_csvs/AML_predicted_celltype_DEG.csv", index_col=0)

In [ ]:
genes = deg_sub[deg_sub.columns[0]].tolist()[:40]

In [ ]:
for gene in genes:
    result = deg_full_data.eq(gene).stack()
    print(22 * "-", gene, 20 * "-")

    if result[result].shape[0] > 0:
        for i in range(7):  # print up to 7 clusters
            row_index, column_index = result[result].index[i]
            values = deg_full_data.loc[
                row_index,
                [
                    column_index.split("_name")[0] + "_pval",
                    column_index.split("_name")[0] + "_logf",
                ],
            ].values
            logfold = values[1]
            pval = values[0]

            if pval < 0.0001 and logfold > 1.2:
                print(np.round(logfold, 2), column_index, np.round(pval, 5))